##  调用训练好的模型 将文本输出成向量，再使用xgboost

In [72]:
import os
import pandas as pd 
import numpy as np
import joblib
import torch
import torch.nn as nn
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed

In [73]:
df = pd.read_parquet('D:/redata/NLP_sen/textdata_cleaned_1-1.parquet')
df['time']=df['time'].dt.strftime('%Y-%m-%d')

idx000985 = 'D:/redata/fields_full/000985.XSHG_hist_index.pkl'

adjclose = 'D:/redata/fields_full/adj_close.pk'

idx = pd.read_pickle(idx000985)
adj_close = pd.read_pickle(adjclose)
adj_return =  adj_close/adj_close.shift(1) - 1
adj_return = adj_return.replace(np.inf,np.nan).replace(-np.inf,np.nan)

df['time'] = pd.to_datetime(df['time'])

In [74]:
df

,report_id,time,stock_code,stock_name,content,classification
0,753935,2016-01-04,603636,南威软件,南威软件设立河北分布云基金政务云业务重大突破事件公司公告与国控张家口鼎众投资河北国控以及中投...,1
1,753946,2016-01-04,300014,亿纬锂能,亿纬锂能动力锂电池进入订单释放期投资要点公司动力电池持续放量是公司未来高速成长的最大驱动力签...,1
2,753974,2016-01-04,300294,博雅生物,博雅生物再次获批两个新浆站血制品业绩弹性大事件江西省卫计委同意公司在赣州市于都县和九江市都昌...,1
3,753991,2016-01-04,002030,达安基因,达安基因体外诊断业领导者打造平台型产业链本报告导读基因测序行业领先的全产业链公司随医疗服务终...,1
4,754015,2016-01-04,600351,亚宝药业,亚宝药业点评报告喜儿康获新版认证外延步伐加快事件月日公司全资子公司亚宝药业太原制药取得对乙酰...,1
...,...,...,...,...,...,...
356842,1447702,2022-04-29,688266,泽璟制药,泽璟制药多纳非尼竞争格局好创新药陆续进入上市阶段核心观点事件年公司实现营收亿元研发费用亿元归...,1
356843,1447703,2022-04-29,000636,风华高科,风华高科业绩小幅下滑行业或将触底回暖事件公司于月日发布年年度报告与年一季报公告显示年公司实现...,1
356844,1447704,2022-04-29,002271,东方雨虹,东方雨虹一季报点评增长保持韧性再显防水龙头实力核心观点收入增长稳健股权激励费用影响业绩公司实...,1
356845,1447680,2022-04-29,000002,万科A,万科一季度盈利稳增建议关注后续拿地节奏业绩回顾年一季度业绩符合我们预期万科公布年一季度业绩营...,1


## beta超额

In [75]:
temp_df = pd.DataFrame(columns=['date', 'ticker', 'classification'])
temp_df['ticker'] = df['stock_code']
temp_df['date'] = df['time'].values
temp_df['classification'] = df['classification'].values
temp_df = temp_df.sort_values(by=['date', 'ticker'])
temp_df.reset_index(drop=True,inplace=True)
temp_df['date'] = pd.to_datetime(temp_df['date'])

start_date = temp_df.index.min()
end_date = temp_df.index.max()
date_range = pd.date_range(start_date, end_date, freq='B')#freq='B'

temp_df = temp_df.drop_duplicates(subset=['date', 'ticker'])
temp_df = temp_df.pivot_table(index='date', columns='ticker', values='classification', aggfunc='first')

In [76]:
adj_return_cp =  adj_return.copy()
adj_return_cp = adj_return_cp[(adj_return_cp.index >= '2016-01-04') & (adj_return_cp.index <= '2022-04-29')]
final_table_idxret = adj_return_cp * temp_df
final_table_idxret['idx_ret'] = np.nan

In [77]:
# 将最后一列的数据类型转换为浮点数
final_table_idxret.iloc[:, -1] = final_table_idxret.iloc[:, -1].astype(float)

# 遍历每一行并计算和
for i in range(0,1706):
    tempdata = final_table_idxret.iloc[i, :-1].mean()
    final_table_idxret.iloc[i, -1] = tempdata

adj_return_cp['idx_ret'] = final_table_idxret['idx_ret']
final_table_idx = adj_return_cp
final_table_idx.replace(0, np.nan, inplace=True)
final_table_idx

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,688788,688789,688793,688798,688799,688800,688819,688981,689009,idx_ret
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.055046,NaN,-0.100000,-0.099602,-0.098175,NaN,-0.099321,-0.100223,-0.099010,-0.099862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.079231
2016-01-05,0.006178,NaN,-0.063665,-0.015487,-0.032755,NaN,0.018850,-0.047030,-0.056044,-0.042081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.010225
2016-01-06,0.011404,NaN,0.012927,0.031461,0.025896,NaN,0.013876,0.036364,0.040745,0.022364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034557
2016-01-07,-0.051171,NaN,-0.100051,-0.099129,-0.100000,NaN,-0.088504,-0.100251,-0.099553,-0.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.076756
2016-01-08,0.016453,NaN,0.006239,0.003628,0.009709,NaN,-0.002002,0.009749,0.001242,0.006944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,-0.075342,-0.049470,-0.099878,-0.051948,-0.084668,-0.044160,-0.067797,-0.100213,-0.100946,-0.098690,...,-0.092267,-0.056324,-0.200036,-0.058615,-0.057215,-0.102681,-0.090573,-0.068817,-0.049688,-0.064298
2022-04-26,-0.008081,0.006904,-0.041272,-0.027397,0.005000,-0.050671,-0.027273,-0.046209,-0.063158,0.004845,...,-0.042084,-0.048553,0.002040,-0.000604,-0.018006,0.025443,-0.036585,-0.030536,-0.011714,-0.021694
2022-04-27,0.062458,-0.038502,0.019760,-0.051643,NaN,0.043956,0.028037,0.100621,0.044944,0.028930,...,0.055447,0.004759,0.016516,0.020239,0.011205,0.047895,0.038397,0.020911,0.011853,0.047518


In [78]:
# 假设 final_table_idx 是一个 DataFrame
df_empty = final_table_idx - final_table_idx
df_empty.replace(0, np.nan, inplace=True)
df_empty = df_empty.drop(columns='idx_ret')
df_empty.index = final_table_idx.index

def calculate_beta(i, r):
    # stock_returns = final_table_idx.iloc[:i, r-2:r-1] 5280 5281
    stock_returns = final_table_idx.iloc[:i, r:r+1]
    market_returns = final_table_idx.iloc[:i, -1]
    
    df_merge = stock_returns.merge(market_returns, on='date', how='left')
    df_merge = df_merge.fillna(0)

    if df_merge.shape[0] == 0:
        return i-1, r, 0
    else:
        y = df_merge.iloc[:, 0].values.reshape(-1, 1)
        x = df_merge.iloc[:, 1].values.reshape(-1, 1)
    
        multi_linear = LinearRegression(fit_intercept=False)  # 无截距项
        multi_linear.fit(x, y)
        beta = multi_linear.coef_[0][0]
        return i-1, r, beta

# 并行计算
# results = Parallel(n_jobs=-1)(delayed(calculate_beta)(i, r) for i in range(1, 1538) for r in range(2, 5282))
results = Parallel(n_jobs=-1)(delayed(calculate_beta)(i, r) for i in range(1, 1538) for r in range(0, 5280))

In [79]:
# 将结果填充到 df_empty
for i, r, beta in results:
    df_empty.iloc[i, r] = beta #这边的 i in range(0, 1537) for r in range(0, 5280))

beta = pd.DataFrame(df_empty)
beta = beta.replace(0, np.nan)
beta = beta.ffill()

In [80]:
beta

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,0.694748,NaN,1.262125,1.257096,1.239097,NaN,1.253553,1.264936,1.249629,1.260386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,0.673469,NaN,1.343446,1.261315,1.271276,NaN,1.202821,1.319560,1.318945,1.307157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,0.619331,NaN,1.190654,1.206006,1.189017,NaN,1.076525,1.277433,1.296899,1.203132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,0.640038,NaN,1.240013,1.243391,1.238801,NaN,1.109999,1.289968,1.296940,1.246741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,0.646784,NaN,1.220804,1.220821,1.223987,NaN,1.083251,1.274088,1.270197,1.228273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,0.508829,0.379710,0.704984,0.722830,0.775275,0.335896,0.593167,1.144201,0.665653,0.875447,...,0.115494,0.149901,0.097835,0.092978,0.075143,0.055481,0.165397,0.151933,0.139048,NaN
2022-04-26,0.508671,0.378904,0.706368,0.723454,0.774112,0.338208,0.593934,1.145340,0.668249,0.874177,...,0.117053,0.151970,0.100309,0.092762,0.075088,0.056376,0.163851,0.153707,0.140514,NaN
2022-04-27,0.513115,0.372345,0.704766,0.713470,0.769843,0.341444,0.593913,1.150702,0.669780,0.872714,...,0.119348,0.157568,0.100308,0.094167,0.077023,0.057366,0.168506,0.157316,0.142166,NaN


In [103]:
final_table = final_table_idx.iloc[:, :5281]
idx_ret = final_table_idx.iloc[:, 5281:5282]

newdf = pd.DataFrame(index = final_table.index,columns = final_table.columns)
# newdf = newdf.reindex(date_range)

for i in range(5280):
    ex = final_table.iloc[:,i] - beta.iloc[:,i] * final_table_idx['idx_ret']
    newdf.iloc[:,i] = ex.values
    
exces_ret = pd.DataFrame(newdf)
exces_ret

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,0.013064,NaN,-0.049929,-0.00259,-0.019757,NaN,0.031148,-0.033538,-0.042558,-0.028716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,-0.009999,NaN,-0.028219,-0.010215,-0.015192,NaN,-0.023325,-0.00778,-0.004072,-0.019212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,-0.002044,NaN,-0.004872,-0.00369,-0.004914,NaN,-0.003304,-0.001237,-0.000004,-0.004305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,0.005241,NaN,-0.014924,-0.017536,-0.011509,NaN,-0.02078,-0.012337,-0.020777,-0.014348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,-0.042626,-0.025055,-0.054549,-0.005472,-0.03482,-0.022562,-0.029657,-0.026644,-0.058146,-0.042401,...,-0.10795,-0.082629,-0.050033,-0.194058,-0.053783,-0.053647,-0.092046,-0.080804,-0.059877,NaN
2022-04-26,0.002954,0.015124,-0.025948,-0.011703,0.021794,-0.043334,-0.014388,-0.021361,-0.048661,0.023809,...,-0.029227,-0.038787,-0.046377,0.004053,0.001025,-0.016783,0.028998,-0.033251,-0.027488,NaN
2022-04-27,0.038075,-0.056195,-0.013729,-0.085546,NaN,0.027731,-0.000184,0.045942,0.013117,-0.01254,...,0.019669,0.04796,-0.000007,0.012041,0.016579,0.00848,0.039888,0.030921,0.014155,NaN


In [36]:
# # # t-1 到 t+1 单只股票的超额
# adj_return_stocks = adj_return.copy()

# # idx['idx000985_return'] = idx['close'].pct_change()
# # idx['date'] = pd.to_datetime(idx['date'])
# # idx = idx[(idx['date']>='2014-01-02')&(idx['date']<'2024-06-06')]

# # adj_return_stocks['idx000985_return'] = idx['idx000985_return'].values

# # start_date = adj_return.index.min()
# # end_date = adj_return.index.max()
# # date_range = pd.date_range(start_date, end_date,)

# newdf = pd.DataFrame(index = adj_close.index,columns = adj_close.columns)
# # newdf = newdf.reindex(date_range)

# for i in range(5281):
#     ex = adj_return_stocks.iloc[:,i] - final_table_idx['idx_ret']
#     newdf.iloc[:,i] = ex.values

# exces_ret = pd.DataFrame(newdf)
# exces_ret

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
return1 = exces_ret
return2 = exces_ret.rolling(3).sum()

return1 = return1.stack().reset_index()
return2 = return2.stack().reset_index()
return1.columns = ['date','ticker','return']
return2.columns = ['date','ticker','return']

final_table = df.merge(return2,how='left',left_on=['time','stock_code'],right_on=['date','ticker'])
final_table = final_table.merge(return1,how='left',left_on=['time','stock_code'],right_on=['date','ticker'])
final_table = final_table.drop(columns = ['date_x','ticker_x','date_y','ticker_y'])
final_table['return']=final_table['return_x'].fillna(final_table['return_y'])
final_table = final_table.drop(columns = ['return_x','return_y']).dropna()

percentiles = final_table['return'].quantile([0.3, 0.7])
final_table['classification'] = pd.cut(final_table['return'], bins=[-float('inf'), percentiles[0.3], percentiles[0.7], float('inf')], labels=[0, 1, 2])
final_table = final_table.reset_index(drop=True)
final_table

C:\Users\sensen.t\AppData\Local\Temp\ipykernel_4048\3027212736.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_table['return']=final_table['return_x'].fillna(final_table['return_y'])


,report_id,time,stock_code,stock_name,content,classification,return
0,753935,2016-01-04,603636,南威软件,南威软件设立河北分布云基金政务云业务重大突破事件公司公告与国控张家口鼎众投资河北国控以及中投...,1,0.000000
1,753946,2016-01-04,300014,亿纬锂能,亿纬锂能动力锂电池进入订单释放期投资要点公司动力电池持续放量是公司未来高速成长的最大驱动力签...,1,0.000000
2,753974,2016-01-04,300294,博雅生物,博雅生物再次获批两个新浆站血制品业绩弹性大事件江西省卫计委同意公司在赣州市于都县和九江市都昌...,1,0.000000
3,753991,2016-01-04,002030,达安基因,达安基因体外诊断业领导者打造平台型产业链本报告导读基因测序行业领先的全产业链公司随医疗服务终...,1,0.000000
4,754015,2016-01-04,600351,亚宝药业,亚宝药业点评报告喜儿康获新版认证外延步伐加快事件月日公司全资子公司亚宝药业太原制药取得对乙酰...,1,0.000000
...,...,...,...,...,...,...,...
324903,1447702,2022-04-29,688266,泽璟制药,泽璟制药多纳非尼竞争格局好创新药陆续进入上市阶段核心观点事件年公司实现营收亿元研发费用亿元归...,1,-0.009656
324904,1447703,2022-04-29,000636,风华高科,风华高科业绩小幅下滑行业或将触底回暖事件公司于月日发布年年度报告与年一季报公告显示年公司实现...,0,-0.033562
324905,1447704,2022-04-29,002271,东方雨虹,东方雨虹一季报点评增长保持韧性再显防水龙头实力核心观点收入增长稳健股权激励费用影响业绩公司实...,0,-0.036228
324906,1447680,2022-04-29,000002,万科A,万科一季度盈利稳增建议关注后续拿地节奏业绩回顾年一季度业绩符合我们预期万科公布年一季度业绩营...,1,-0.012033


In [105]:
# return1 = (adj_close/adj_close.shift(1)-1)
# return2 = (adj_close.shift(-1)/adj_close.shift(1)-1)

# start_date = return1.index.min()
# end_date = return1.index.max()
# date_range = pd.date_range(start_date, end_date,)
# # 补全日期
# return1 = return1.reindex(date_range).bfill()
# return1 = return1.stack().reset_index()
# return2 = return2.stack().reset_index()
# return1.columns = ['date','ticker','return']
# return2.columns = ['date','ticker','return']

# # percentiles = final_table['return'].quantile([0.3, 0.7])
# # final_table['classification'] = pd.cut(final_table['return'], bins=[-float('inf'), percentiles[0.3], percentiles[0.7], float('inf')], labels=[-1, 0, 1])
# # final_table = final_table[final_table['stock_code'].str[:3].isin(['000', '001', '002', '003', '600', '601', '603', '605'])]
# final_table.loc[final_table['return'] < -0.0,'classification'] = 0
# final_table.loc[final_table['return'] == 0.0,'classification'] = 1
# final_table.loc[final_table['return'] > 0.0,'classification'] = 2
# final_table = final_table.reset_index(drop=True)
# final_table

In [106]:
final_table.to_parquet('D:/redata/NLP_sen/final_table.parquet')

In [107]:
final_table = pd.read_parquet('D:/redata/NLP_sen/final_table.parquet')

In [108]:
# 指定保存模型的文件路径
save_path = 'D:/redata/NLP_sen/3classmodel500pt.pth'
# save_path = 'D:/redata/NLP_sen/test/3classmodel.pth'

# 加载模型
model = torch.load(save_path, map_location=torch.device('cpu'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [109]:
# 加载预训练的配置
# 加载分词器
config = BertConfig.from_pretrained('D:/redata/NLP_sen/finbert/bert_config.json')
tokenizer = BertTokenizer.from_pretrained('D:/redata/NLP_sen/finbert/vocab.txt')

model.to(device)
model.eval()  # 设置模型为评估模式


# 准备数据
# 假设你的数据已经被处理成输入文本和标签的形式
input_texts = final_table['content']
input_ids = []
attention_masks = []

for text in input_texts:
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

input_ids = torch.cat(input_ids, dim=0).to(device)
attention_masks = torch.cat(attention_masks, dim=0).to(device)
dataset = TensorDataset(input_ids, attention_masks)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

result = []
with torch.no_grad():
    for batch in dataloader:
        input_ids_batch, attention_masks_batch = batch
        input_ids_batch, attention_masks_batch = input_ids_batch.to(device), attention_masks_batch.to(device)

        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        result.append(outputs.hidden_states[-1][:, 0, :].cpu().numpy())

C:\Users\sensen.t\Anaconda3\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:2185: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [110]:
print(tokenizer.decode(input_ids[-1]))

[CLS] 中 科 电 气 盈 利 能 力 迎 来 拐 点 加 速 扩 张 高 速 成 长 投 资 要 点 公 司 发 布 年 年 报 及 年 一 季 报 公 司 全 年 实 现 营 业 收 入 亿 元 同 比 实 现 归 母 净 利 润 亿 元 同 比 扣 非 后 归 母 净 利 润 亿 元 同 比 业 绩 符 合 预 期 年 公 司 实 现 营 业 收 入 亿 元 同 比 实 现 归 母 净 利 润 亿 元 同 比 扣 非 后 实 现 归 母 净 利 润 亿 元 同 比 业 绩 超 预 期 季 度 环 比 保 持 增 长 盈 利 能 力 迎 来 拐 点 分 季 度 来 看 公 司 实 现 营 收 亿 元 环 比 实 现 营 收 亿 元 环 比 在 产 能 释 放 及 负 极 成 本 压 力 向 下 传 导 的 推 动 下 公 司 营 收 逐 季 攀 升 盈 利 能 力 来 看 公 司 盈 利 能 力 实 现 触 底 反 弹 毛 利 率 修 复 至 我 们 认 为 这 主 要 是 受 益 于 成 本 压 力 传 导 以 及 石 墨 化 自 供 提 升 石 墨 化 预 付 款 拖 累 公 司 现 金 流 公 司 [SEP]


In [111]:
input_ids[-1]

tensor([ 101,  704, 4906, 4510, 3698, 4659, 1164, 5543, 1213, 6816, 3341, 2866,
        4157, 1217, 6862, 2810, 2476, 7770, 6862, 2768, 7270, 2832, 6598, 6206,
        4157, 1062, 1385, 1355, 2357, 2399, 2399, 2845, 1350, 2399,  671, 2108,
        2845, 1062, 1385, 1059, 2399, 2141, 4385, 5852,  689, 3119, 1057,  783,
        1039, 1398, 3683, 2141, 4385, 2495, 3678, 1112, 1164, 3883,  783, 1039,
        1398, 3683, 2807, 7478, 1400, 2495, 3678, 1112, 1164, 3883,  783, 1039,
        1398, 3683,  689, 5327, 5016, 1394, 7564, 3309, 2399, 1062, 1385, 2141,
        4385, 5852,  689, 3119, 1057,  783, 1039, 1398, 3683, 2141, 4385, 2495,
        3678, 1112, 1164, 3883,  783, 1039, 1398, 3683, 2807, 7478, 1400, 2141,
        4385, 2495, 3678, 1112, 1164, 3883,  783, 1039, 1398, 3683,  689, 5327,
        6631, 7564, 3309, 2108, 2428, 4384, 3683,  924, 2898, 1872, 7270, 4659,
        1164, 5543, 1213, 6816, 3341, 2866, 4157, 1146, 2108, 2428, 3341, 4692,
        1062, 1385, 2141, 4385, 5852, 31

In [112]:
attention_masks[-1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [113]:
# 二次训练模型得到的768维向量
feature = np.concatenate(result,axis=0)

try:
    save_dir = 'D:/redata/NLP_sen/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, '3class_xgboost_feature_500pt.npy')
    np.save(save_path, feature)
    print(f"Feature saved successfully to {save_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Feature saved successfully to D:/redata/NLP_sen/3class_xgboost_feature_500pt.npy


In [114]:
# 定义XGBoost的label和features 
feature = np.load('D:/redata/NLP_sen/3class_xgboost_feature_500pt.npy')
label = np.array(final_table['classification'])
np.save('D:/redata/NLP_sen/3class_xgboost_label_500pt.npy',label)
label = np.load('D:/redata/NLP_sen/3class_xgboost_label_500pt.npy')
label = label + 1

# path1 = 'Z:/NPL_sen/feature_org.npy'
# path2 = 'Z:/NPL_sen/xgboost_label.npy'

# feature = np.load(path1)
# label = np.load(path2)
# label = label + 1

In [115]:
feature_copy = feature.copy()
feature_copy = pd.DataFrame(feature_copy)
feature_copy

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.660593,-1.533324,-0.254349,0.654513,0.403565,-0.693365,-0.909980,0.699016,-1.212633,1.188309,...,-0.960861,-1.093518,-0.250591,-0.031099,-0.008387,-0.556574,-0.890893,-0.042687,-0.119558,-0.665102
1,1.517722,0.149247,0.220937,-1.437236,0.532118,-0.523754,-1.863817,1.229543,0.119555,-0.332688,...,-0.750942,-1.339011,-0.048055,1.113288,0.543836,0.036857,-0.774697,-0.019106,0.121740,-0.335500
2,1.276700,-1.215384,0.664773,-0.050148,0.016063,-0.835794,-0.519707,-0.744273,-0.327459,0.083680,...,-1.123371,-1.087749,-0.835365,0.253101,1.406389,-1.336075,-0.268989,1.030165,1.488263,-0.578157
3,0.449712,-0.514733,1.911903,-0.320150,-0.048760,-0.913266,-0.311187,-0.299888,-1.689907,0.127128,...,-2.084758,-0.874323,0.705025,0.287448,0.312258,-0.959606,-1.288787,0.629211,-0.277913,-1.288332
4,-0.212849,-0.931231,0.619994,0.172244,-0.327848,-0.858479,1.044566,-0.416220,-0.664763,1.081543,...,-1.273426,-0.260944,-0.935841,-0.699666,0.001166,-0.255806,-0.411538,0.388315,0.563625,-0.866180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324903,0.750198,-0.419800,-0.144098,-0.818761,0.455857,-0.784439,-0.251335,0.231520,-0.606057,-0.290433,...,-0.600555,-0.054315,-1.347445,-0.017506,0.078619,-0.824124,-0.908886,-0.504786,1.849795,-0.396826
324904,0.479197,-0.315095,-1.478325,0.025121,0.572096,-0.118213,1.478951,-0.549588,1.101304,0.391990,...,1.154235,0.437787,-0.867607,-1.182790,0.248058,-0.486173,-0.238603,-1.088463,1.482201,0.672713
324905,0.792208,-0.415368,0.516298,-0.676201,1.037129,-0.600679,0.210859,-0.073112,-1.125206,-1.040805,...,-0.203298,-0.753490,0.068859,0.356399,0.603403,-0.054563,0.986828,-1.075728,0.574365,-0.509393
324906,0.387300,-1.194243,-0.197552,-0.178888,-0.406022,-0.115009,1.008447,0.362457,-0.634886,-1.155354,...,0.094082,0.452472,-0.076717,0.278252,0.770593,-0.995081,-0.609220,-0.338615,-0.033405,-0.922299


In [116]:
label_copy = label.copy()
label_copy = pd.DataFrame(label_copy)
label_copy

,0
0,2
1,2
2,2
3,2
4,2
...,...
324903,2
324904,1
324905,1
324906,2
